In [1]:
from legion.toolchain.metrics import Metric
from legion.toolchain import model

In [2]:
model.init('test summation', '1.0')

In [3]:
def calculate_sum(x):
    return int(x['a']) + int(x['b'])


def calculate_sum_and_pow(x):
    return 10 * (x['a'] + x['b'])

In [4]:
model.send_metric(Metric.TEST_ACCURACY, 100)

In [5]:
model.show_local_metrics()

,Model ID,Model Version,Last Update,test-accuracy
0,test-summation,1.0,05/19/19 12:08:41,100


In [6]:
model.export_untyped(lambda x: {'result': int(calculate_sum(x))})

model.export(lambda x: {'result': int(calculate_sum_and_pow(x))}, {
    'a': model.int32,
    'b': model.int32
}, endpoint='sum_and_pow')

model.save()

In [7]:
model.show_local_metrics()

,Model ID,Model Version,Last Update,test-accuracy
0,test-summation,1.0,05/19/19 12:08:41,100
